<a href="https://colab.research.google.com/github/gunGeongun/yolo-webapp/blob/main/YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics fastapi uvicorn python-multipart pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/2

In [2]:
!pip install -U opencv-python pillow numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.


In [3]:
!yolo task=detect mode=predict model=yolov8n.pt


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 6.25M/6.25M [00:00<00:00, 373MB/s]
WARNING ⚠️ 'source' argument is missing. Using default 'source=/usr/local/lib/python3.11/dist-packages/ultralytics/assets'.
Ultralytics 8.3.137 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

image 1/2 /usr/local/lib/python3.11/dist-packages/ultralytics/assets/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 38.6ms
image 2/2 /usr/local/lib/python3.11/dist-packages/ultralytics/assets/zidane.jpg: 384x640 2 persons, 1 tie, 46.2ms
Speed: 9.0ms preprocess, 42.4ms inference, 172.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/

In [4]:
!mkdir -p www/static/css
!mkdir -p www/static/js
!mkdir -p www/templates


In [5]:
%%writefile app.py
from fastapi import FastAPI, UploadFile, File, Form, Request
from fastapi.responses import JSONResponse, HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import numpy as np
import cv2
from ultralytics import YOLO
import os
import base64
from PIL import Image
from io import BytesIO
import pathlib

# 디렉토리 구조 확인
current_dir = pathlib.Path.cwd()
print(f"현재 작업 디렉토리: {current_dir}")
if not os.path.exists("www"):
    os.makedirs("www")
if not os.path.exists("www/templates"):
    os.makedirs("www/templates")

# FastAPI 앱 생성
app = FastAPI()

# CORS 미들웨어 설정 (모든 오리진 허용)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 정적 파일 및 템플릿 설정
app.mount("/static", StaticFiles(directory="www/static"), name="static")
templates = Jinja2Templates(directory="www/templates")

# YOLOv8 모델 로드
model = YOLO("yolov8n.pt")

@app.get("/")
async def read_root(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

@app.get("/api/info")
async def api_info():
    return {"message": "YOLOv8 Object Detection API"}

@app.post("/api/detect")
async def detect_objects(image: UploadFile = File(...)):
    # 이미지 읽기
    contents = await image.read()
    nparr = np.frombuffer(contents, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # YOLOv8로 객체 감지
    results = model(img)
    result = results[0]

    # 결과 처리
    detected_objects = []
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])
        cls_name = result.names[cls_id]

        detected_objects.append({
            "class": cls_name,
            "confidence": round(conf, 2),
            "bbox": [x1, y1, x2, y2]
        })

    # 바운딩 박스가 있는 이미지 생성
    for obj in detected_objects:
        x1, y1, x2, y2 = obj["bbox"]
        cls_name = obj["class"]
        conf = obj["confidence"]

        # 바운딩 박스 그리기
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # 레이블 텍스트
        label = f"{cls_name} {conf:.2f}"

        # 텍스트 배경
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
        cv2.rectangle(img, (x1, y1 - 20), (x1 + w, y1), (0, 255, 0), -1)

        # 텍스트 추가
        cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    # 이미지를 base64로 인코딩
    _, buffer = cv2.imencode('.jpg', img)
    img_base64 = base64.b64encode(buffer).decode('utf-8')

    return {
        "detected_objects": detected_objects,
        "processed_image": f"data:image/jpeg;base64,{img_base64}"
    }

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=3000)


Writing app.py


In [6]:
%%writefile www/templates/index.html
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>YOLOv8 객체 인식</title>
    <link rel="stylesheet" href="/static/css/styles.css">
</head>
<body>
    <div id="app"></div>

    <!-- React 및 ReactDOM 로드 -->
    <script src="https://cdnjs.cloudflare.com/ajax/libs/react/18.2.0/umd/react.production.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/react-dom/18.2.0/umd/react-dom.production.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/babel-standalone/7.21.5/babel.min.js"></script>

    <!-- 애플리케이션 코드 -->
    <script type="text/babel" src="/static/js/app.js"></script>
</body>
</html>


Writing www/templates/index.html


In [7]:
%%writefile www/static/css/styles.css
body {
    font-family: Arial, sans-serif;
    max-width: 100%;
    margin: 0;
    padding: 10px;
    background-color: #f5f5f5;
}

h1 {
    text-align: center;
    color: #333;
}

#app {
    display: flex;
    flex-direction: column;
    align-items: center;
}

#controls {
    margin-bottom: 10px;
    display: flex;
    flex-wrap: wrap;
    justify-content: center;
    gap: 10px;
}

button {
    padding: 8px 16px;
    background-color: #4CAF50;
    color: white;
    border: none;
    border-radius: 4px;
    cursor: pointer;
    font-size: 14px;
}

button:hover {
    background-color: #45a049;
}

button:disabled {
    background-color: #cccccc;
    cursor: not-allowed;
}

#video-container {
    position: relative;
    margin-bottom: 15px;
    max-width: 100%;
}

video {
    max-width: 100%;
    border: 1px solid #ddd;
    border-radius: 4px;
}

#canvas-overlay {
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;
    height: 100%;
}

#result-image {
    max-width: 100%;
    border: 1px solid #ddd;
    border-radius: 4px;
    margin-bottom: 15px;
}

#detection-results {
    width: 100%;
    max-width: 600px;
    margin-bottom: 15px;
    padding: 10px;
    background-color: white;
    border: 1px solid #ddd;
    border-radius: 4px;
}

.status {
    text-align: center;
    margin: 10px 0;
    color: #666;
}

.object-item {
    margin-bottom: 5px;
    padding: 5px;
    background-color: #f9f9f9;
    border-radius: 4px;
}

.loading {
    text-align: center;
    font-style: italic;
    color: #888;
}


Writing www/static/css/styles.css


In [8]:
%%writefile www/static/js/app.js
const { useState, useEffect, useRef } = React;

function App() {
    const [cameraMode, setCameraMode] = useState('environment'); // 기본값은 후면 카메라
    const [isStreaming, setIsStreaming] = useState(false);
    const [detectedObjects, setDetectedObjects] = useState([]);
    const [processedImage, setProcessedImage] = useState(null);
    const [isProcessing, setIsProcessing] = useState(false);
    const [errorMessage, setErrorMessage] = useState(null);

    const videoRef = useRef(null);
    const canvasRef = useRef(null);
    const streamRef = useRef(null);
    const intervalRef = useRef(null);

    // 카메라 시작
    const startCamera = async () => {
        try {
            if (streamRef.current) {
                streamRef.current.getTracks().forEach(track => track.stop());
            }

            const constraints = {
                video: {
                    facingMode: cameraMode,
                    width: { ideal: 640 },
                    height: { ideal: 480 }
                },
                audio: false
            };

            const stream = await navigator.mediaDevices.getUserMedia(constraints);
            streamRef.current = stream;

            if (videoRef.current) {
                videoRef.current.srcObject = stream;
                videoRef.current.play();
                setIsStreaming(true);
                setErrorMessage(null);
                startDetectionInterval();
            }
        } catch (err) {
            console.error("카메라 접근 오류:", err);
            setErrorMessage(`카메라 접근 오류: ${err.message}`);
            setIsStreaming(false);
        }
    };

    // 카메라 중지
    const stopCamera = () => {
        if (streamRef.current) {
            streamRef.current.getTracks().forEach(track => track.stop());
            streamRef.current = null;
        }

        if (intervalRef.current) {
            clearInterval(intervalRef.current);
            intervalRef.current = null;
        }

        setIsStreaming(false);
        setDetectedObjects([]);
        setProcessedImage(null);
    };

    // 카메라 모드 변경
    const toggleCameraMode = () => {
        const newMode = cameraMode === 'environment' ? 'user' : 'environment';
        setCameraMode(newMode);

        if (isStreaming) {
            stopCamera();
            setTimeout(startCamera, 300); // 잠시 대기 후 재시작
        }
    };

    // 주기적 객체 감지 시작
    const startDetectionInterval = () => {
        if (intervalRef.current) {
            clearInterval(intervalRef.current);
        }

        intervalRef.current = setInterval(() => {
            detectObjects();
        }, 10000); // 10초마다 객체 감지
    };

    // 이미지 캡처 및 객체 감지
    const detectObjects = async () => {
        if (!videoRef.current || !isStreaming || isProcessing) return;

        setIsProcessing(true);

        try {
            // 캔버스에 비디오 프레임 캡처
            const video = videoRef.current;
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            const ctx = canvas.getContext('2d');
            ctx.drawImage(video, 0, 0, canvas.width, canvas.height);

            // 이미지를 Blob으로 변환
            const blob = await new Promise(resolve => {
                canvas.toBlob(resolve, 'image/jpeg', 0.8);
            });

            // FormData 생성
            const formData = new FormData();
            formData.append('image', blob, 'capture.jpg');

            // API 요청
            const response = await fetch('/api/detect', {
                method: 'POST',
                body: formData,
            });

            if (!response.ok) {
                throw new Error(`HTTP 오류: ${response.status}`);
            }

            const data = await response.json();
            setDetectedObjects(data.detected_objects);
            setProcessedImage(data.processed_image);
            setErrorMessage(null);
        } catch (err) {
            console.error("객체 감지 오류:", err);
            setErrorMessage(`객체 감지 오류: ${err.message}`);
        } finally {
            setIsProcessing(false);
        }
    };

    // 마운트 시 이벤트 리스너 설정
    useEffect(() => {
        // 컴포넌트 언마운트 시 정리
        return () => {
            stopCamera();
        };
    }, []);

    return (
        <div>
            <h1>YOLOv8 모바일 객체 인식</h1>

            <div id="controls">
                {!isStreaming ? (
                    <button onClick={startCamera} disabled={isProcessing}>
                        카메라 시작
                    </button>
                ) : (
                    <button onClick={stopCamera} disabled={isProcessing}>
                        카메라 중지
                    </button>
                )}

                <button onClick={toggleCameraMode} disabled={isProcessing}>
                    {cameraMode === 'environment' ? '전면 카메라로 전환' : '후면 카메라로 전환'}
                </button>

                {isStreaming && (
                    <button onClick={detectObjects} disabled={isProcessing}>
                        지금 감지하기
                    </button>
                )}
            </div>

            {errorMessage && (
                <div className="status" style={{ color: 'red' }}>
                    {errorMessage}
                </div>
            )}

            <div id="video-container">
                <video ref={videoRef} width="640" height="480" autoPlay playsInline muted></video>
                <canvas ref={canvasRef} id="canvas-overlay"></canvas>
            </div>

            {isProcessing && (
                <div className="loading">처리 중...</div>
            )}

            {processedImage && (
                <div>
                    <h2>감지된 이미지</h2>
                    <img id="result-image" src={processedImage} alt="감지된 객체" />
                </div>
            )}

            <div id="detection-results">
                <h2>감지된 객체</h2>
                {detectedObjects.length > 0 ? (
                    <div>
                        <p><strong>감지된 객체 수:</strong> {detectedObjects.length}</p>
                        {detectedObjects.map((obj, index) => (
                            <div key={index} className="object-item">
                                <strong>{obj.class}</strong> (신뢰도: {obj.confidence.toFixed(2)})
                                <br />
                                <small>위치: x1={obj.bbox[0]}, y1={obj.bbox[1]}, x2={obj.bbox[2]}, y2={obj.bbox[3]}</small>
                            </div>
                        ))}
                    </div>
                ) : (
                    <p>감지된 객체가 없습니다.</p>
                )}
            </div>

            <div className="status">
                {isStreaming ? '카메라가 활성화되었습니다. 10초마다 자동으로 객체를 감지합니다.' : '카메라가 비활성화되었습니다.'}
            </div>
        </div>
    );
}

ReactDOM.render(<App />, document.getElementById('app'));

Writing www/static/js/app.js


In [9]:
from pyngrok import ngrok
ngrok.set_auth_token("2wQlf6kY0pLngmwKXXpGEnS9sh1_4rGe73ogcu1pgG5VMNC8e")
print("ngrok 인증 토큰이 설정되었습니다.")

ngrok 인증 토큰이 설정되었습니다.


In [10]:
%%writefile run_server.py
import subprocess
import time
from pyngrok import ngrok

# FastAPI 서버 시작
server_process = subprocess.Popen(["python", "app.py"])
print("FastAPI 서버가 시작되었습니다.")

# ngrok 터널 생성
http_tunnel = ngrok.connect(3000)
print(f"ngrok 터널이 생성되었습니다: {http_tunnel.public_url}")

try:
    # 앱이 계속 실행되도록 대기
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    # 종료 시 프로세스 정리
    server_process.terminate()
    ngrok.kill()

# 이 코드는 Colab이 계속 실행 중일 때만 작동합니다

Writing run_server.py


In [11]:
!python run_server.py

FastAPI 서버가 시작되었습니다.
ngrok 터널이 생성되었습니다: https://7555-34-87-6-235.ngrok-free.app
현재 작업 디렉토리: /content
INFO:     Started server process [1401]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:3000 (Press CTRL+C to quit)
INFO:     211.231.103.106:0 - "GET / HTTP/1.1" 200 OK
INFO:     221.139.219.182:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     221.139.219.182:0 - "GET /apple-touch-icon.png HTTP/1.1" 404 Not Found
INFO:     221.139.219.182:0 - "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 Not Found
INFO:     221.139.219.182:0 - "GET / HTTP/1.1" 200 OK
INFO:     221.139.219.182:0 - "GET /static/css/styles.css HTTP/1.1" 200 OK
INFO:     221.139.219.182:0 - "GET /static/js/app.js HTTP/1.1" 200 OK

0: 640x480 1 person, 1 laptop, 41.9ms
Speed: 4.2ms preprocess, 41.9ms inference, 193.9ms postprocess per image at shape (1, 3, 640, 480)
INFO:     221.139.219.182:0 - "POST /api/detect HTTP/1.1" 200 OK

0